# Modelo de Identificação de Fraudes

In [3]:
# Importando o pandas

import pandas as pd

In [4]:
# Importando a base

transacoes = pd.read_csv('creditcard.csv')

<a id='modelo'></a>
### Criando um modelo de aprendizado de máquinas para identificar fraude
- Vamos criar um modelo para identificar fraude sem fazer nenhum tratamento nessa base

**Podemos ajustar as colunas Time e Amount para que elas fiquem entre 0 e 1**

In [5]:
# Primeiro para a coluna Time

transacoes.Time = transacoes.Time / transacoes.Time.max()

In [6]:
# E então para a coluna Amount

transacoes.Amount = transacoes.Amount / transacoes.Amount.max()

In [7]:
# Separando X e y

X = transacoes.drop('Class',axis=1)
y = transacoes.Class

**Separando em treino e teste**

In [8]:
# Separando em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### Realizando o random undersampling

In [9]:
# Importando o RandomUnderSampler do imblearn

from imblearn.under_sampling import RandomUnderSampler 

In [10]:
# Definindo o RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

In [11]:
# Definindo a nova amostra

X_resRUS, y_resRUS = rus.fit_resample(X_train, y_train)

**<font color='blue'>Com Support Vector Machines (SVC)**

In [12]:
# Fazendo o fit

from sklearn.svm import SVC

clf_SVC = SVC(random_state=0,probability=True).fit(X_resRUS, y_resRUS)

In [13]:
# Fazendo a previsão

y_pred_SVC = clf_SVC.predict(X_test)

y_pred_proba_SVC = clf_SVC.predict_proba(X_test)[:,1]

In [14]:
# Visualizando a matriz de confusão

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred_SVC)

array([[92611,  1214],
       [   22,   140]], dtype=int64)

In [15]:
# Traçando a área sobre a curva precisão x recall

from sklearn import metrics

precision_SVC,recall_SVC,thresholds_SVC = metrics.precision_recall_curve(y_test,y_pred_proba_SVC)

print(metrics.auc(recall_SVC, precision_SVC))

0.6847172261858195


**E novamente utilizar o GridSearchCV**

In [16]:
# Importando o GridSearchCV

from sklearn.model_selection import GridSearchCV

In [17]:
# Utilizando os parâmetros do SVC

parametros = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

parametros2 = {'C': [0.001, 0.1, 0.01, 1, 10, 100, 1000], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['scale', 'auto']}

In [18]:
# Selecionando a SVC

SVC_GS = SVC(random_state=0,probability=True)

In [19]:
# Criando um novo classificador usando os parâmetros que escolhemos anteriormente

clf_GS = GridSearchCV(SVC_GS, parametros, scoring='recall')

clf_GS2 = GridSearchCV(SVC_GS, parametros2, scoring='recall')

In [20]:
# Fazendo o fit dos nossos dados

clf_GS.fit(X_resRUS, y_resRUS)

clf_GS2.fit(X_resRUS, y_resRUS)

GridSearchCV(estimator=SVC(probability=True, random_state=0),
             param_grid={'C': [0.001, 0.1, 0.01, 1, 10, 100, 1000],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='recall')

In [21]:
# Visualizando os melhores parâmetros definidos pelo GridSearchCV 

print(clf_GS.best_params_)
print(clf_GS2.best_params_)

{'C': 10, 'kernel': 'linear'}
{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}


In [22]:
# Usando esse modelo para fazer as previsões

y_pred_SVC_GS = clf_GS.predict(X_test)
y_pred_SVC_GS2 = clf_GS2.predict(X_test)

In [23]:
# Analisando a matriz de confusão

print(confusion_matrix(y_test,y_pred_SVC_GS))

print(confusion_matrix(y_test,y_pred_SVC_GS2))

[[90816  3009]
 [   16   146]]
[[85791  8034]
 [   12   150]]


**<font color='blue'>Random Forest**

In [34]:
# Fazendo o fit

from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier(max_depth=2, random_state=0).fit(X_resRUS, y_resRUS)

In [35]:
# Fazendo a previsão

y_pred_RF = clf_RF.predict(X_test)

y_pred_proba_RF = clf_RF.predict_proba(X_test)[:,1]

In [36]:
# Visualizando a matriz de confusão

confusion_matrix(y_test,y_pred_RF)

array([[92977,   848],
       [   25,   137]], dtype=int64)

In [37]:
# Utilizando os parâmetros do Random Forest

parametros_RF = {'n_estimators': [40, 50, 100, 200], 'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [2,3,4,5,6,7,8,9]}

In [38]:
# Selecionando o Random Forest

RF_GS = RandomForestClassifier(random_state=0)

In [39]:
# Criando um novo classificador usando os parâmetros que escolhemos anteriormente

clf_RF_GS = GridSearchCV(RF_GS, parametros_RF, scoring='recall')

In [41]:
# Fazendo o fit dos nossos dados

clf_RF_GS.fit(X_resRUS, y_resRUS)

GridSearchCV(estimator=RandomForestClassifier(random_state=0),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [40, 50, 100, 200]},
             scoring='recall')

In [42]:
# Visualizando os melhores parâmetros definidos pelo GridSearchCV 

clf_RF_GS.best_params_

{'criterion': 'gini', 'max_depth': 9, 'n_estimators': 40}

In [43]:
# Usando esse modelo para fazer as previsões

y_pred_RF_GS = clf_RF_GS.predict(X_test)

In [44]:
# Analisando a matriz de confusão

confusion_matrix(y_test,y_pred_RF_GS)

array([[91599,  2226],
       [   19,   143]], dtype=int64)

In [47]:
# Calculando o Recall

from sklearn.metrics import recall_score

recall_score(y_test, y_pred_RF_GS)

0.8827160493827161